## 패키지 가져오기

In [1]:
from pykrx import stock
from pykrx import bond
import pandas_datareader.data as web
import FinanceDataReader as fdr

from talib import RSI, BBANDS, MACD

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import seaborn as sns

import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint
from sklearn.model_selection import train_test_split

from sklearn.metrics import adjusted_mutual_info_score
from sklearn import cluster, covariance, manifold


from datetime import datetime
from dateutil.relativedelta import relativedelta
import time
from tqdm import tqdm

import multiprocessing as mp
from multiprocessing import Pool, Manager

import warnings
warnings.filterwarnings('ignore')

In [2]:
num_cores = mp.cpu_count()
print(f"사용가능한 코어 수: {num_cores}")

사용가능한 코어 수: 8


## 전략 수립 및 구현

2015년-2016년의 수익률을 기준으로 공적분 검정을 통해 ($\alpha$ = 0.05) pair를 찾고  
pair에서 저평가된 주식을 매수, 고평가된 주식을 매도  
이 때, 저평가와 고평가의 여부는 페어로 묶인 두 주식의 ratio의 z-score를 사용   
(ratio의 z-score를 구할 때, ratio의 평균은 22영업일(1개월) 이동평균을, 분산은 22영업일(1개월) 이동분산을 사용함)  
평균에서 1 $\sigma$ 초과(또는 미만)가 되는 지의 여부를 기준으로 함.  
한편 2 $\sigma$ 초과(또는 미만)이 되는 경우, breakout을 사용해 비중이 0이 되도록 함.  
비중 설정 방식은 동일 가중 방식을 사용.  
이 때, 2015년~2016년 동안 계속해서 코스피에 상장되어있던 주식을 대상으로 함  
2017년 1월-2023년 1월의 기간동안, 리밸런싱은 1일마다 실시.  

In [3]:
def return_price_df(tickers, start_date, end_date):
    df = pd.DataFrame()
    for ticker in tqdm(tickers):
        price_df = stock.get_market_ohlcv(start_date, end_date, ticker)['종가'].to_frame(ticker)
        df = pd.concat([df, price_df], axis=1)
    return df

In [4]:
price_df = pd.read_csv('pair_trading_price_df.csv', index_col=0)
price_df.index = pd.to_datetime(price_df.index, format='%Y-%m-%d')
price_df = price_df[price_df.index >= "2015-01-01"]
price_df = price_df.dropna(axis=1)
return_df = price_df.pct_change().dropna()
return_df.head()

,003070,011160,003415,102280,123700,009970,023350,114090,002250,013520,...,001040,028050,047050,00781K,107590,18064K,073240,029780,000145,105560
날짜,,,,,,,,,,,,,,,,,,,,,
2015-01-05,-0.002913,-0.010310,-0.072319,0.044213,-0.005739,-0.001085,0.001030,0.013678,-0.033835,0.006270,...,0.012699,-0.034568,-0.013158,0.006944,0.018389,0.003617,-0.025536,-0.056000,0.014706,-0.016667
2015-01-06,-0.008919,-0.058246,0.005376,-0.013699,0.001443,0.001086,-0.013374,-0.026987,-0.019455,0.015576,...,-0.006270,-0.042977,-0.023333,-0.008046,-0.014749,-0.003604,-0.049266,-0.027845,0.000000,-0.002825
2015-01-07,0.002948,-0.036581,0.063280,-0.013889,-0.005764,0.000000,0.010428,-0.009245,-0.015873,0.008180,...,0.000000,-0.003026,-0.001706,-0.003476,0.003323,-0.018163,0.026461,-0.004981,-0.010033,-0.008499
2015-01-08,-0.004332,0.009264,-0.010059,0.000000,0.007246,0.011931,0.005160,0.052877,0.000000,0.000000,...,0.034698,0.001542,0.003419,-0.001163,-0.001673,-0.014814,-0.011815,0.020025,-0.013514,0.045714
2015-01-09,0.007303,0.011248,-0.033023,0.019206,0.012950,0.027867,-0.004107,0.060561,0.048387,0.010649,...,0.015248,0.037470,0.017036,-0.004657,-0.013304,0.018777,0.011957,-0.002454,-0.010274,0.006831


In [5]:
def find_cointegrated_pairs(data, significance=0.05):
    n = data.shape[1]
    pvalue_matrix = np.zeros((n,n))
    keys = data.columns
    pairs = []
    for i in tqdm(range(n)):
        for j in range(i+1, n):
            result = coint(data[keys[i]], data[keys[j]])
            pvalue_matrix[i, j] = result[1]
            if result[1] < significance:
                pairs.append((keys[i], keys[j]))
    return pvalue_matrix, pairs

In [6]:
pvalues, pairs = find_cointegrated_pairs(return_df, 0.05)
all_tickers = []
for pair in pairs:
    all_tickers.append(pair[0])
    all_tickers.append(pair[1])
all_tickers = list(set(all_tickers))
print("페어의 개수: ", len(pairs))
print("페어에 속한 주식의 개수: ", len(all_tickers))

100%|██████████| 856/856 [1:54:23<00:00,  8.02s/it]  


페어의 개수:  365256
페어에 속한 주식의 개수:  856


In [7]:
start_date, end_date = "2016-12-01", "2023-01-31"
pair_price_df = return_price_df(all_tickers, start_date, end_date)
pair_price_df = pair_price_df.fillna(0)
pair_price_df.to_csv("pair_price_df(coint).csv")

100%|██████████| 856/856 [01:11<00:00, 12.00it/s]


In [9]:
pair_price_df = pd.read_csv("pair_price_df(coint).csv", index_col=0)
pair_price_df.index = pd.to_datetime(pair_price_df.index, format='%Y-%m-%d')

In [11]:
pair_daily_return = pd.DataFrame()

for i in tqdm(range(len(pairs))):
    ticker1, ticker2 = pairs[i]
    temp_df = pair_price_df.loc[:, [ticker1, ticker2]]

    fwd_return = temp_df.pct_change().shift(-1)
    fwd_return = fwd_return.fillna(0)
    fwd_return.index = pd.to_datetime(fwd_return.index, format='%Y-%m-%d')

    ratio = temp_df[ticker1] / temp_df[ticker2]
    rolling_mean = ratio.rolling(window=22).mean()
    rolling_std = ratio.rolling(window=22).std()
    z_ratio = ((ratio-rolling_mean)/rolling_std).dropna()
    signal_df = pd.DataFrame()
    signal1 = ((1 < z_ratio) & (z_ratio< 2)).astype(int)
    signal2 = ((-2 < z_ratio) & (z_ratio< -1)).astype(int)
    signal_df[ticker1] = signal2 - signal1
    signal_df[ticker2] = signal1 - signal2

    daily_return = fwd_return.mul(signal_df).sum(axis=1)
    pair_daily_return[f'pair_{i}'] = daily_return

  0%|          | 608/365256 [00:22<3:45:29, 26.95it/s]


KeyboardInterrupt: 

In [ ]:
def calculate_daily_return(pair_daily_return):
    nonzero = (pair_daily_return != 0).sum(axis=1)
    daily_return = pair_daily_return.sum(axis=1).div(nonzero).fillna(0)
    return daily_return
daily_return = calculate_daily_return(pair_daily_return).to_frame("daily_return")
daily_return

In [ ]:
cumulative_return = (np.exp(np.log(daily_return['daily_return']+1).cumsum())-1).to_frame("cumulative_return")
cumulative_return

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111)
ax.plot(cumulative_return)
ax.set_title("CUMULATIVE RETURN", fontsize=16)
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

## 자세한 결과 분석

In [ ]:
import pyfolio as pf
from pyfolio.plotting import plot_rolling_returns, plot_rolling_sharpe
from pyfolio.timeseries import forecast_cone_bootstrap

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(16, 8))

plot_rolling_returns(daily_return['daily_return'],
                     logy=False,
                     legend_loc='best',
                     volatility_match=False,
                    ax=axes[0])
plot_rolling_sharpe(daily_return['daily_return'], ax=axes[1], rolling_window=63)
axes[0].set_title('Cumulative Returns', fontsize=16)
axes[1].set_title('Rolling Sharpe Ratio (3 Months)', fontsize=16)

In [ ]:
pf.create_returns_tear_sheet(daily_return['daily_return'])